In [1]:
import tensorflow as tf
from keras.models import Model
from keras.layers import Input, Dense, LSTM, multiply, concatenate, Activation, Masking, Reshape
from keras.layers import Conv1D, BatchNormalization, GlobalAveragePooling1D, Permute, Dropout
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [2]:
import numpy as np
import math
from utils.generic_utils import load_dataset_at

from utils.constants import MAX_NB_VARIABLES, NB_CLASSES_LIST, MAX_TIMESTEPS_LIST 

In [3]:
X_train, Y_train, X_test, Y_test, is_timeseries = load_dataset_at(0, fold_index=None, normalize_timeseries=False) 
X_test = pad_sequences(X_test, maxlen=MAX_NB_VARIABLES[0], padding='post', truncating='post')
Y_test = to_categorical(Y_test, len(np.unique(Y_test)))

Loading train / test dataset :  ../cut_data/ ../cut_data/
Finished processing train dataset..
Finished loading test dataset..

Number of train samples :  1847 Number of test samples :  793
Number of classes :  7
Sequence length :  506


In [4]:
def get_model(weights = None):
    #Squeeze excitation block

    def squeeze_excite_block(input):
        filters = input._keras_shape[-1] # channel_axis = -1 for TF
        se = GlobalAveragePooling1D()(input)
        se = Reshape((1, filters))(se)
        se = Dense(filters // 16,  activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
        se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)
        se = multiply([input, se])
        return se


    ip = Input(shape=(MAX_NB_VARIABLES[0], MAX_TIMESTEPS_LIST[0]))

    x = Masking()(ip)
    x = LSTM(8)(x)
    x = Dropout(0.2)(x)

    y = Permute((2, 1))(ip)
    y = Conv1D(64, 5, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = squeeze_excite_block(y)

    y = Conv1D(128, 4, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = squeeze_excite_block(y)

    y = Conv1D(64, 3, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)

    y = GlobalAveragePooling1D()(y)

    x = concatenate([x, y])

    out = Dense(NB_CLASSES_LIST[0], activation='softmax')(x)

    model = Model(inputs=ip, outputs=out)

    if(not weights == None):
        model.load_weights(weights)

    return model

In [5]:
model = get_model('./model_weights931890.h5')

W0522 00:07:54.682206 11188 deprecation_wrapper.py:119] From E:\Anaconda3 2019.03\envs\tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0522 00:07:54.709207 11188 deprecation_wrapper.py:119] From E:\Anaconda3 2019.03\envs\tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0522 00:07:54.727198 11188 deprecation_wrapper.py:119] From E:\Anaconda3 2019.03\envs\tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0522 00:07:55.035021 11188 deprecation.py:323] From E:\Anaconda3 2019.03\envs\tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:2974: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a 

In [6]:
#Check for accuracy
Y_pred = model.predict(X_test)

t = []
for i in range(len(Y_pred)):
    t.append(sum((max(Y_pred[i])==Y_pred[i]) * Y_test[i]) == 0)
1-sum(t)/len(Y_pred)

0.8890290037831021

In [7]:
#For the time being latent space has the same shape as X_shape
latent_vector_shape = (2, 506)
X_shape = (2, 506)
Y_shape = (7,)

In [10]:
#MAKE graph
sess_autoZoom = tf.InteractiveSession()
'''
    Construct tf-Graph
''' 

CONST_LAMBDA = 10000
k = 0.00
x = tf.placeholder(tf.float32, (None,) + X_shape, name='X') #Input data
y = tf.placeholder(tf.float32, (None,) + Y_shape, name='Y')    # Output
adv = tf.placeholder(tf.float32, (None,) + latent_vector_shape, name='adv') #avdersarial example

# compute loss
new_x = adv + x
output = model(new_x)

l2_norm = tf.reduce_sum(tf.square(new_x - x), axis=[1,2])

real = tf.reduce_sum(y * output, axis=1)
other = tf.reduce_max((1 - y) * output, axis=1)
    
loss1 = CONST_LAMBDA * tf.maximum(tf.log(real + 1e-30) - tf.log(other + 1e-30), -k)
loss2 = l2_norm

loss_batch = loss1 + loss2
loss = tf.reduce_sum(loss_batch) # sum over all the batch samples

# # initialize variables and load target model
sess_autoZoom.run(tf.global_variables_initializer())

#weights are reset
model.load_weights('./model_weights931890.h5')

In [11]:
success_count = 0
query_summary = {}
adv_summary = {}
fail_count = 0
invalid_count = 0

q = 1 
b = q
#Iterate for each test example
for t in range(X_test.shape[0]):

    print("\n start attacking target", t, "...")
       
    mt = 0               # accumulator m_t in Adam
    vt = 0               # accumulator v_t in Adam

    beta1 = 0.9            # parameter beta_1 in Adam
    beta2 = 0.999          # parameter beta_2 in Adam
    learning_rate = 2e-3   # learning rate in Adam
    
    batch_size = 1        # batch size
    MAX_ITE = 2000        # maximum number of iterations

    #For the time being it has the same shape as X
    init_adv = np.zeros((1,) + latent_vector_shape)           # initial adversarial perturbation

    X = np.expand_dims(X_test[t], 0)           # target sample X
    Y = np.expand_dims(Y_test[t], 0)           # target sample's lable Y
    
    # check if (X, Y) is a valid target, y checking if it is classified correctly
    Y_pred = model.predict(X)
    if sum((max(Y_pred[0]) == Y_pred[0]) * Y[0]) == 0:
        print("not a valid target.")
        invalid_count += 1
        continue
    
    # when performing blackbox attack, we feed forward X = [X + old_adv, X + new_adv] in the same batch for estimating the gradient
    X = np.repeat(X, 2, axis=0)
    Y = np.repeat(Y, 2, axis=0)

    var_size = init_adv.size
    beta = 1/(var_size)

    # main loop for the optimization
    for epoch in range(1, MAX_ITE + 1):
        #Using random vector gradient estimation 

        #random noise
        u = np.random.normal(loc=0, scale=1000, size = (q, var_size))
        u_mean = np.mean(u, axis=1, keepdims=True)
        u_std = np.std(u, axis=1, keepdims=True)
        u_norm = np.apply_along_axis(np.linalg.norm, 1, u, keepdims=True)
        u = u/u_norm

        #For estimation of F(x + beta*u) and F(x)
        var = np.concatenate( (init_adv, init_adv + beta * u.reshape((q,)+ (latent_vector_shape)) ), axis=0)
        losses, scores = sess_autoZoom.run([loss_batch, output], feed_dict={adv: var, x: X, y: Y}) 
        

        #Gradient estimation
        grad = np.zeros((q, var_size), dtype = np.float32)
        for j in range(q):
            grad[j] = (b * (losses[j + 1] - losses[0])* u[j]) / beta
        
        avg_grad = np.mean(grad, axis=0)

        # ADAM update
        mt = beta1 * mt + (1 - beta1) * avg_grad
        vt = beta2 * vt + (1 - beta2) * (avg_grad * avg_grad)
        corr = (np.sqrt(1 - np.power(beta2,epoch))) / (1 - np.power(beta1, epoch))

        m = init_adv .reshape(-1)
        m -= learning_rate * corr * (mt / (np.sqrt(vt) + 1e-8))

        #update the adversarial example
        init_adv = m.reshape(init_adv.shape)


        if epoch == MAX_ITE:
            print("attack failed!")
            fail_count += 1
            break

        if sum((scores[0] == max(scores[0]))*Y[0])==0:
            print(f"attack successed! with iter {epoch}")
            success_count += 1
            query_summary[t] = epoch
            adv_summary[t] = init_adv
            break


 start attacking target 0 ...
attack successed! with iter 1519

 start attacking target 1 ...


KeyboardInterrupt: 